# AuxTel mount issue - 25-Feb-2022

In this notebook, investigate mount issue from 20220215\
Why was the mount issued a "full stop"?

In [ ]:
import sys, time, os, asyncio

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import pandas as pd
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler

In [ ]:
from lsst_efd_client import EfdClient
from lsst_efd_client import  __version__ as efdVersion
print(efdVersion)

In [ ]:
# Get EFD client and the butler
client = EfdClient('ldf_stable_efd')
butler = Butler('/repo/main', collections="LATISS/raw/all")

In [ ]:
for i, expId in enumerate(range(2022021500194, 2022021500279)):
    mData = butler.get('raw.metadata', detector=0, exposure=expId)
    date_beg = Time(mData['DATE-BEG'], format='isot', scale='tai')
    if i > 0:
        delta = TimeDelta(date_beg.utc - last_date_beg.utc)
        print(i, expId, delta.sec)
    last_date_beg = date_beg

In [ ]:
expStart = 2022021500194
expEnd = 2022021500245
indices = []
begs = []
ends = []
bads = []
for i, expId in enumerate(range(expStart, expEnd)):
    index = expId - 2022021500000
    indices.append(index)
    mData = butler.get('raw.metadata', detector=0, exposure=expId)
    date_beg = Time(mData['DATE-BEG'], format='isot', scale='tai')
    date_end = Time(mData['DATE-END'], format='isot', scale='tai')
    if i == 0:
        tstart = date_beg.utc
    begs.append(date_beg.utc)
    ends.append(date_end.utc)
    if index in [200,206,218,224,230,236,242,262]:
        bads.append(date_beg.utc)
tstop = date_end.utc    

In [ ]:
az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", 
                                            "azimuthCalculatedAngle",  tstart, tstop)

In [ ]:
stopTracking = await client.select_time_series("lsst.sal.ATMCS.command_stopTracking", 
                                            "value",  tstart, tstop)

In [ ]:
fig = plt.figure(figsize = (8,4))
plt.suptitle(f"Mount Tracking - ExpIds {expStart} - {expEnd}", fontsize = 18)
ax1 = az['azimuthCalculatedAngle'].plot(legend=True, color='red')
for i in range(len(begs)):
    ax1.axvline(begs[i].to_datetime(), ymin=0.1, ymax=0.25, color="blue")
ax1.axvline(begs[i].to_datetime(), ymin=0.1, ymax=0.25, color="blue", label="Shutter open")    
for j in range(len(bads)):
    ax1.axvline(bads[j].to_datetime(), ymin=0.27, ymax=0.42, color="green")  
ax1.axvline(bads[j].to_datetime(), ymin=0.27, ymax=0.42, color="green", label="Bad tracking")      
for k in range(len(stopTracking)):
    ax1.axvline(stopTracking.index[k], ymin=0.44, ymax=0.59, color="magenta")
ax1.axvline(stopTracking.index[k], ymin=0.44, ymax=0.59, color="magenta", label="StopTracking")    
ax1.set_ylim(179.5, 180.5)
ax1.legend(loc='upper right')
plt.savefig("/project/cslage/AuxTel/mount_graphs/Mount_Fails_15Feb22.pdf")